# 📌 Transform: Depois Enriquecer

[Clique para acessar os dados](https://docs.google.com/spreadsheets/d/1YQhDNSt1sOrzq6AKxyYG6wzRAvM-M7l7jJRIHunNB88/copy?gid=1190375422#gid=1190375422)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_receitas = pd.read_csv('/content/Cópia de Dados - TCC - Planilhas - Receitas.csv')

## 1️⃣ Ao nível de colunas

### 💡 Coluna Condicional ou Personalizada

In [ ]:
df_receitas.columns

Index(['atendimento_id', 'data_lancamento', 'receita_codigo',
       'venda_quantidade', 'venda_valor', 'cliente_id', 'cliente_sexo',
       'cliente_nascimento_data', 'cliente_origem',
       'profissional_especialidade', 'pagamento_data', 'pagamento_forma'],
      dtype='object')

🔹 Se tivermos uma coluna de data de nascimento, podemos calcular a idade subtraindo do ano atual. Diferente das planilhas não é uma questão de subtração simples, porque data não é tratada como número

In [ ]:
from datetime import datetime

# Criando a coluna de idade baseada na data de nascimento. Pegamos o ano atual e subtraímos o ano de nascimento, obtendo a idade.
df_receitas["cliente_idade"] = datetime.today().year - pd.to_datetime(df_receitas["cliente_nascimento_data"]).dt.year

In [ ]:
df_receitas["cliente_idade"].value_counts()

,count
cliente_idade,
43.0,59
35.0,48
36.0,48
32.0,46
34.0,43
...,...
84.0,1
78.0,1
14.0,1


🔹 Podemos dividir a idade em faixas numéricas predefinidas com pd.cut(). O pd.cut() considera o limite inferior fechado e o superior aberto, exceto o último intervalo, que é fechado.

In [ ]:
# Criando categorias de idade com pd.cut()

df_receitas["cliente_faixa_etaria"] = pd.cut(
    df_receitas["cliente_idade"],
    bins=[0, 17, 35, 60, 100],  # Limites das faixas etárias
    labels=["0-17", "18-35", "36-60", "61-100"]
)

In [ ]:
df_receitas["cliente_faixa_etaria"].value_counts(dropna=False)

,count
cliente_faixa_etaria,
36-60,580
18-35,422
61-100,130
NaN,103
0-17,7


🔹 Existem muitos caminhos. Poderíamos ter usado o np.select novamente para isso

In [ ]:
# Criando condições para as faixas etárias
condicoes = [
    df_receitas["cliente_idade"] < 18,
    (df_receitas["cliente_idade"] >= 18) & (df_receitas["cliente_idade"] <= 35),
    (df_receitas["cliente_idade"] > 35) & (df_receitas["cliente_idade"] <= 60),
    df_receitas["cliente_idade"] > 60
]

# 📌 Definindo os rótulos correspondentes
categorias = ["0-17", "18-35", "36-60", "61-100"]

# 📌 Criando a nova coluna com np.select()
df_receitas["cliente_faixa_etaria"] = np.select(condicoes, categorias, default="Desconhecido")


In [ ]:
df_receitas["cliente_faixa_etaria"].value_counts(dropna=False)

,count
cliente_faixa_etaria,
36-60,580
18-35,422
61-100,130
Desconhecido,103
0-17,7


Isso é o que chamamos de **feature engeniier ou engenharia de features**, uma habilidade especialmente importante

### 💡 Aplicando funções

No entanto, pode ser que a criação de features necessite de algum cálculo mais complexo. Um método muito usado na manipulação de dados é aplicar uma função nos dados ou em parte deles, para isso o método <font color = "orange">**`.apply()`</font>** é muito utilizado.

Importante dizer que esse método **aplica uma determinada função em apenas uma coluna ou linha do DF.**

In [ ]:
# Função para categorizar a idade
def classificar_faixa_etaria(idade):
    if idade < 18:
        return "0-17"
    elif idade <= 35:
        return "18-35"
    elif idade <= 60:
        return "36-60"
    elif idade > 60:
        return "61-100"
    else:
        return "Desconhecido"

In [ ]:
# Aplicando a função com apply()
df_receitas["cliente_faixa_etaria"] = df_receitas["cliente_idade"].apply(classificar_faixa_etaria)

**Comparações**

| Critério                       | `np.select()`                                           | `apply()` com função                                    |
| ------------------------------ | ------------------------------------------------------- | ------------------------------------------------------- |
| 🔄 **Velocidade**              | ✅ Mais rápido para grandes volumes de dados             | ❌ Mais lento (loop interno por linha)                   |
| 🧠 **Clareza e lógica**        | ❌ Pode ser menos legível com muitas condições complexas | ✅ Mais legível, fácil de entender                       |
| 🧩 **Flexibilidade**           | ❌ Limitado a expressões booleanas simples               | ✅ Pode usar qualquer lógica Python                      |
| 🛠️ **Engenharia de features** | ✅ Ideal para regras simples de classificação            | ✅ Ideal para regras complexas e condicionais encadeadas |
| 🔍 **Depuração e testes**      | ❌ Mais difícil de testar e isolar erros                 | ✅ Função independente facilita testes                   |
